<a href="https://colab.research.google.com/github/crystalloide/RAG/blob/main/LAB36_Agentic_AI_Tools_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LAB36 : Qu’est-ce qui définit un outil en IA agentique ?

**Objectif :** Apprendre à concevoir, implémenter et exposer un outil qu’un agent peut appeler.

**Durée estimée :** 10 à 15 minutes

**Livrable :** Un script Python fonctionnel définissant 2 à 3 outils et montrant comment un LLM peut les invoquer.

## Étape 1 : Qu’est-ce qu’un outil ? (Concept)

Un **Tool** ou **outil** est toute fonction externe que l’agent peut appeler pour étendre ses capacités.

**Exemples :**

- Calculatrice
- Recherche Web
- Requête de base de données
- Appel d’API

**Format :** nom + description + fonction appelable

Le LLM choisit quand invoquer l’outil en fonction des instructions.

## Étape 2 : Configuration

Installation des bibliothèques requises :

In [3]:
!pip install -q langchain langchain-openai python-dotenv requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.8/84.8 kB 2.9 MB/s eta 0:00:00


**Configuration de la clé API OpenAI :**


In [2]:
import os
from google.colab import userdata

# Récupérer la clé API depuis les secrets Colab
# Pour ajouter : cliquez sur 🔑 dans le panneau de gauche
try:
    openai_api_key = userdata.get('OPENAI_API_KEY')
    os.environ['OPENAI_API_KEY'] = openai_api_key
    print("✓ Clé API OpenAI chargée depuis les secrets Colab")
except:
    print("⚠ Secrets Colab non configurés. Veuillez ajouter OPENAI_API_KEY.")
    print("Instructions : Cliquez sur 🔑 dans le panneau gauche > Ajouter un nouveau secret")

✓ Clé API OpenAI chargée depuis les secrets Colab


## Étape 3 : Définir des outils simples :

Création de 3 outils de démonstration :

1. **Calculator** - Évaluation d'expressions mathématiques
2. **Joke Generator** - Raconter des blagues sur un sujet donné
3. **Weather** - Obtenir des informations météorologiques fictives pour une ville (à l’aide de l’API OpenMeteo)

In [5]:
from langchain_core.tools import Tool
import requests
import math

# 1. Calculator
def calculator(expr: str):
    """Evaluate a mathematical expression."""
    try:
        return str(eval(expr))
    except Exception as e:
        return f"Error: {e}"

# 2. Joke generator
def joke(topic: str):
    """Generate a joke about a given topic."""
    return f"Here's a {topic}-themed joke: Why did the {topic} cross the road? To learn Agentic AI!"

# 3. Weather (demo API call using Open-Meteo)
def weather(city: str):
    """Get demo weather information for a city."""
    url = f"https://geocoding-api.open-meteo.com/v1/search?name={city}"
    try:
        r = requests.get(url, timeout=10).json()
        if "results" not in r or len(r["results"]) == 0:
            return f"No weather info for {city}"
        coords = r["results"][0]
        return f"{city} located at lat {coords['latitude']}, lon {coords['longitude']} (demo response)."
    except Exception as e:
        return f"Error fetching weather: {e}"

# Define tools list
tools = [
    Tool(
        name="Calculator",
        func=calculator,
        description="Evaluate math expressions. Input should be a valid Python math expression as a string."
    ),
    Tool(
        name="Joke",
        func=joke,
        description="Tell a quick joke about a topic. Input should be a topic name as a string."
    ),
    Tool(
        name="Weather",
        func=weather,
        description="Get demo weather info for a city. Input should be a city name as a string."
    ),
]

print("✓ Tools defined successfully")
print(f"Available tools: {[t.name for t in tools]}")

✓ Tools defined successfully
Available tools: ['Calculator', 'Joke', 'Weather']


**Test the tools individually:**

In [6]:
# Test de chaque Tool :
print("Calculator test:", calculator("12 * (7+3)"))
print("Joke test:", joke("robot"))
print("Weather test:", weather("Paris"))

Calculator test: 120
Joke test: Here's a robot-themed joke: Why did the robot cross the road? To learn Agentic AI!
Weather test: Paris located at lat 48.85341, lon 2.3488 (demo response).


## Étape 4 : Création d'un agent "intelligent" :

Nous allons maintenant créer un agent capable de choisir de lui-même l’outil à utiliser en fonction de la requête de l’utilisateur.

In [13]:
import langchain
print(langchain.__version__)

1.2.7


In [15]:
from langchain import agents
print(dir(agents))

['AgentState', '__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', 'create_agent', 'factory', 'middleware', 'structured_output']


In [18]:
import inspect
from langchain.agents import create_agent
print(inspect.signature(create_agent))

(model: 'str | BaseChatModel', tools: 'Sequence[BaseTool | Callable[..., Any] | dict[str, Any]] | None' = None, *, system_prompt: 'str | SystemMessage | None' = None, middleware: 'Sequence[AgentMiddleware[StateT_co, ContextT]]' = (), response_format: 'ResponseFormat[ResponseT] | type[ResponseT] | dict[str, Any] | None' = None, state_schema: 'type[AgentState[ResponseT]] | None' = None, context_schema: 'type[ContextT] | None' = None, checkpointer: 'Checkpointer | None' = None, store: 'BaseStore | None' = None, interrupt_before: 'list[str] | None' = None, interrupt_after: 'list[str] | None' = None, debug: 'bool' = False, name: 'str | None' = None, cache: 'BaseCache[Any] | None' = None) -> 'CompiledStateGraph[AgentState[ResponseT], ContextT, _InputAgentState, _OutputAgentState[ResponseT]]'


In [19]:
from langchain_openai import ChatOpenAI
from langchain.agents import create_agent

# Initialize the LLM
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Create the agent with system prompt
agent = create_agent(
    model=llm,
    tools=tools,
    system_prompt="You are a helpful assistant. Use the tools available to answer questions accurately."
)

print("✓ Agent initialized successfully")

✓ Agent initialized successfully


#### Informations complémentaires :
Cette version utilise la nouvelle API de **create_agent** avec les paramètres nommés (model=, tools=, system_prompt=) qui correspondent à la signature obtenue.

## Étape 5 : Exécution des requêtes :

Testons l’agent avec différents types de requêtes et observons comment il choisit l’outil approprié.

**👉 Remarque :**

- Le LLM choisit un outil lorsqu’il le juge nécessaire.

- Il n'y a pas besoin de coder en dur la logique ; c’est l’agent qui décide.


### Requête 1 : Calcul mathématique

In [32]:
response = agent.invoke({"messages": [{"role": "user", "content": "What is 12 * (7+3)?"}]})

print("\n" + "="*50)
print("Résultat Final:", response)
print("="*50)
print("\n")
print(response['messages'][-1].content)


Résultat Final: {'messages': [HumanMessage(content='What is 12 * (7+3)?', additional_kwargs={}, response_metadata={}, id='36ec7899-b47a-4a64-a998-a60ab4db3828'), AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 160, 'total_tokens': 183, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_6c0d1490cb', 'id': 'chatcmpl-D5qsaKPVWtXy6teYUxZEe2qqR9ZpX', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c2d58-2d4a-7561-b025-02e10c53de61-0', tool_calls=[{'name': 'Calculator', 'args': {'__arg1': '12 * (7 + 3)'}, 'id': 'call_AG5YeI5yj87l0ZhXmEYZ4CSe', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_token

### Requête 2 : Demander une blague

In [33]:
# Test avec les blagues
response = agent.invoke({"messages": [{"role": "user", "content": "Tell me a joke about robots"}]})

print("\n" + "="*50)
print("Résultat Final:", response)
print("="*50)
print("\n")
print(response['messages'][-1].content)





Résultat Final: {'messages': [HumanMessage(content='Tell me a joke about robots', additional_kwargs={}, response_metadata={}, id='76a38fb4-bfd5-49c7-87a5-e5259aa4a8f0'), AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 156, 'total_tokens': 172, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_6c0d1490cb', 'id': 'chatcmpl-D5qshW4b9D5I08uIgExfKYvxhJOXM', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c2d58-4a56-7f81-a4bc-a6cd6b2f5ec9-0', tool_calls=[{'name': 'Joke', 'args': {'__arg1': 'robots'}, 'id': 'call_BltHogWs4w8obHltoy4oOtZY', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 

### Requête 3 : Informations météorologiques

In [34]:
# Test avec la météo
response = agent.invoke({"messages": [{"role": "user", "content": "What's the weather in Paris?"}]})

print("\n" + "="*50)
print("Résultat Final:", response)
print("="*50)
print("\n")
print(response['messages'][-1].content)


Résultat Final: {'messages': [HumanMessage(content="What's the weather in Paris?", additional_kwargs={}, response_metadata={}, id='5ec7a84c-d9d6-48b7-84ce-98589b92c56a'), AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 156, 'total_tokens': 172, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_6c0d1490cb', 'id': 'chatcmpl-D5qsnkRlvlf1msEfkqoXxQ1uzQcYV', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c2d58-64ff-7cb1-9adc-26ee25fe6307-0', tool_calls=[{'name': 'Weather', 'args': {'__arg1': 'Paris'}, 'id': 'call_3Fmu5zAslcUCWWep0VNN45Km', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens

### Requête 4 : Requête complexe nécessitant un raisonnement

In [36]:
response = agent.invoke({"messages": [{"role": "user", "content": "Calculate 25 squared, then tell me a joke about math."}]})

print("\n" + "="*50)
print("Résultat Final:", response)
print("="*50)
print("\n")
print(response['messages'][-1].content)


Résultat Final: {'messages': [HumanMessage(content='Calculate 25 squared, then tell me a joke about math.', additional_kwargs={}, response_metadata={}, id='ae018d63-dafe-4592-a6ec-d182ebce1735'), AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 50, 'prompt_tokens': 163, 'total_tokens': 213, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_6c0d1490cb', 'id': 'chatcmpl-D5quQWrEqUfkYhrIgpLukJtxzEYdp', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c2d59-efbe-7023-8843-6fb5b84734c6-0', tool_calls=[{'name': 'Calculator', 'args': {'__arg1': '25**2'}, 'id': 'call_Dvaj8R3uPPe4Wj1j4YENTc2A', 'type': 'tool_call'}, {'name': 'Joke', 'args': 

## Étape 6 : Mini-projet (optionnel) :

**Défi :** Ajoutez votre propre outil personnalisé à l’agent !

Voici quelques idées :

1. **search_github** : Rechercher des dépôts GitHub (utiliser l’API GitHub)

2. **summarize_url** : Récupérer et résumer une page web

3. **convert_currency** : Convertir des devises

4. **random_fact** : Afficher un fait aléatoire d’une liste

5. **text_stats** : Compter les mots, les caractères et les phrases d’un texte


```
```
  **Modèle pour créer un nouvel outil :**


In [38]:
from langchain_core.tools import Tool

# Example: Text statistics tool
def text_stats(text: str):
    """Analyze text and return statistics."""
    words = len(text.split())
    chars = len(text)
    sentences = text.count('.') + text.count('!') + text.count('?')
    return f"Text stats: {words} words, {chars} characters, {sentences} sentences."

# Add the new tool
new_tool = Tool(
    name="TextStats",
    func=text_stats,
    description="Analyze text and return statistics (word count, character count, sentence count). Input should be a text string."
)

# Create updated tools list
extended_tools = tools + [new_tool]

# Reinitialize agent with new tool using create_agent
extended_agent = create_agent(
    model=llm,
    tools=extended_tools,
    system_prompt="You are a helpful assistant. Use the tools available to answer questions accurately."
)

print("✓ Extended agent created with new tool")
print(f"Available tools: {[t.name for t in extended_tools]}")

✓ Extended agent created with new tool
Available tools: ['Calculator', 'Joke', 'Weather', 'TextStats']


**Test de l'agent étendu :**

In [47]:
response = extended_agent.invoke({"messages": [{"role": "user", "content": "List the tools you have at your disposal.'"}]})

print("\n" + "="*50)
print("Résultat Final:", response)
print("="*50)
print("\n")
print(response['messages'][-1].content)


Résultat Final: {'messages': [HumanMessage(content="List the tools you have at your disposal.'", additional_kwargs={}, response_metadata={}, id='821a11c4-579e-4d5b-9a3a-dfaae6db3337'), AIMessage(content='I have the following tools at my disposal:\n\n1. **Calculator**: Evaluates math expressions.\n2. **Joke**: Tells a quick joke about a specified topic.\n3. **Weather**: Provides demo weather information for a specified city.\n4. **TextStats**: Analyzes text and returns statistics such as word count, character count, and sentence count.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 79, 'prompt_tokens': 202, 'total_tokens': 281, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_6c0d1490cb', 'id

In [46]:
response = extended_agent.invoke({"messages": [{"role": "user", "content": "Analyze this text and give the tools you haved used if any : 'Agentic AI is amazing! It can use tools. This is the future.'"}]})

print("\n" + "="*50)
print("Résultat Final:", response)
print("="*50)
print("\n")
print(response['messages'][-1].content)



Résultat Final: {'messages': [HumanMessage(content="Analyze this text and give the tools you haved used if any : 'Agentic AI is amazing! It can use tools. This is the future.'", additional_kwargs={}, response_metadata={}, id='223beb56-11bb-4f2e-98c5-afdccbd4bf59'), AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 224, 'total_tokens': 255, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_6c0d1490cb', 'id': 'chatcmpl-D5rRFHNTXfbxMzBX9kXcWORjf9Hfm', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c2d78-fb06-7ae0-986b-c542f279d333-0', tool_calls=[{'name': 'TextStats', 'args': {'__arg1': 'Agentic AI is amazing! It 

### À vous de jouer ! 🚀

Créez votre propre outil personnalisé ci-dessous :

In [ ]:
# TODO: Définir ici la fonction de votre outil personnalisé
def my_custom_tool(input_param: str):
    """Description of what your tool does."""
    # Your implementation here
    return "Your result"

# TODO: Create the Tool object
# my_tool = Tool(
#     name="MyTool",
#     func=my_custom_tool,
#     description="Description permettant à l'agent de comprendre quand utiliser cet outil."
# )

# TODO: Ajouter aux outils et tester !

## Conclusion :

Dans ce TP, nous avons (re)vu :

✅ Ce qu'est un outil dans le contexte de l'IA agentique

✅ Comment définir des outils à l'aide de la classe `Tool` de LangChain

✅ Comment créer un agent capable de sélectionner et d'utiliser intelligemment des outils

✅ Comment l'agent utilise ReAct (Reasoning + Acting) pour résoudre des problèmes

✅ Comment étendre les capacités d'un agent avec des outils personnalisés

**Points clés :**

- Les outils étendent les capacités de l'agent au-delà de la simple génération de langage
- L'agent décide de manière autonome quel outil utiliser en fonction de la requête
- Les outils doivent avoir des noms et des descriptions clairs pour être compris par le LLM
- Plusieurs outils peuvent être combinés pour résoudre des tâches complexes

**Prochaines étapes :**

- Expérimenter avec des outils plus complexes (API, bases de données, opérations sur les fichiers)

- Essayer différents types d'agents (conversationnel, chat structuré, etc.)

- Créer une application concrète combinant plusieurs outils

- Explorer la gestion des erreurs et la validation des outils